# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели.
Можно использовать данные из семинара или любые другие (можно брать только часть текста, если считается слишком долго). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг \<start>  
    - можете использовать тот же подход с матрицей вероятностей, но по строкам хронить биграмы, а по колонкам униграммы
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так)
    - у вас будут словари с индексами биграммов и униграммов, не перепутайте их при переводе индекса в слово - словарь биграммов будет больше словаря униграммов и все индексы из униграммного словаря будут формально подходить для словаря биграммов (не будет ошибки при id2bigram[unigram_id]), но маппинг при этом будет совершенно неправильным

In [2]:
pip install razdel

In [250]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML
from scipy.sparse import lil_matrix, csr_matrix, csc_matrix
from collections import Counter
from nltk.tokenize import sent_tokenize

In [251]:
news = open('lenta.txt').read()

In [252]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

def trigrammer(tokens, n=3):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

def bigrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i +n]))
    return ngrams

In [253]:
sentences_news = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news[:5000000])]

In [254]:
unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    trigrams_news.update(trigrammer(sentence))
    bigrams_news.update(bigrammer(sentence))

In [255]:
matrix_news = lil_matrix((len(bigrams_news),
                        len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}

id2bigram_news = list(bigrams_news)
bigram2id_news = {word:i for i, word in enumerate(bigrams_news)}


for ngram in trigrams_news:
    word1, word2, word3 = ngram.split()
    bigram = word1 + ' ' + word2
    matrix_news[bigram2id_news[bigram], word2id_news[word3]] =  (trigrams_news[ngram]/
                                                                     bigrams_news[bigram])

In [256]:
def generate(matrix, id2word, bigram2id, n=100, start='<start> <start>'):
    text = []
    current = start

    for i in range(n):
        bigram_id = bigram2id[current]
        chosen_word_idx = np.random.choice(matrix.shape[1], p=matrix[bigram_id].toarray()[0])
        word = id2word[chosen_word_idx]
        text.append(word)
        # строку ниже надо было перенести сюда относительно кода из семинара, иначе была ошибка по вероятностям
        current = current.split()[1] + ' ' + word

        if word == '<end>':
            current = '<start> <start>'
    return ' '.join(text)

In [257]:
print(generate(matrix_news, id2word_news, bigram2id_news).replace('<end>', '\n'))

правительственная комиссия во главе с генералом пришли солдаты и офицеры подразделения армии ветераны войны представители штаба подчеркнули что их будет лариса кривцова с 7.40 до 8.00 ежедневно 
 русская версия odigo интегрирована с поисковой системой rambler заглавная страница которой является general tire северная каролина 
 напомним что в 1999 году после победы лейбористской партии министр образования россии филиппов академик-секретарь отделения литературы и языка российской академии наук армении депутат микаел котанян 
 по словам министра руководство страны пойдет на выборах в марте прошлого года и шестого по значимости уровня падения за день сгорает 300 тонн муки учебники медикаменты и необходимая техника


In [258]:
print(generate(matrix_news, id2word_news, bigram2id_news).replace('<end>', '\n'))

все мы хотим об этом корр 
 были воссозданы цветные витражи станции новослободская арочные фрески станции киевская на эскалаторах французские дизайнеры воссоздали светильники станции университет 
 новый автомат еще не могут оправдать удары по боевикам и российским властям 
 в церемонии прощания приняли участие 1500 человек 
 как сообщает агентство прайм-тасс 
 в этой тюрьме под контроль но осталась напряженной утром 6 октября объявили о самороспуске 
 по данным некоторых экспертов днем рождения 
 говорят что им удастся сделать чип передающий хотя бы часть денег развивая сотрудничество с фирмами собственниками которых являются люди чьи имена содержатся в списке потенциальных неудачников


In [259]:
print(generate(matrix_news, id2word_news, bigram2id_news).replace('<end>', '\n'))

кроме того офер нимроди входит в систему слишком много желающих встретиться с президентом рф власти ижевска предложили ввести в заблуждение с помощью специально обученных собак которые будут готовы выложить круглую сумму за вычетом затрат на производство самодельные взрывные устройства 
 предполагается что президент клинтон отправил в отставку премьер-минстра страны массимо д алемы были найдены в кассовом зале вокзала 
 исходя из их суммарной величины 1 декабря в белгард министр обороны филиппин орландо меркардо сообщил журналистам министр сельского хозяйства курс молдавского лея остается стабильным ставки по ним было реализовано 87 тысяч транзакций проведенных через эти счета подорвали репутацию службы 
 всего


In [260]:
def perplexity(logp, N):
    return np.exp((-1/N) * logp)


def compute_join_trigram(text, bigram_counts, trigram_counts):
    prob = 0
    tokens = normalize(text)
    for ngram in trigrammer(['<start>', '<start>'] + tokens + ['<end>']):
        word1, word2, word3 = ngram.split()
        bigram = word1 + ' ' + word2
        if bigram in bigram_counts and ngram in trigram_counts:
            prob += np.log(trigram_counts[ngram]/bigram_counts[bigram])
        else:
            prob += np.log(2e-5)

    return prob, len(tokens)

In [261]:
phrase = 'Спешу сообщить, что на Манежной площади открылась главная Рождественская ярмарка Петербурга. \
В волшебном городе "Конфетобурге" каждый посетитель найдет занятие по душе — здесь можно прокатиться на аттракционах, \
посетить ремесленные и кулинарные мастер-классы и, конечно же, встретиться с Дедом Морозом. \
Самых маленьких гостей ждут двухъярусная карусель, каток и паровозик", — написал он.'
log_prob, N = compute_join_trigram(phrase, bigrams_news, trigrams_news)
perplexity(log_prob, N)

50000.00000000027

In [262]:
phrase = 'Гуляния на Московской площади пройдут с 22 декабря по 8 января. Тематикой новой площадки станут сказки \
Александра Сергеевича Пушкина. Здесь ожидается культурно-образовательная программа для гостей, а также зимние игры и развлечения. По традиции также можно будет приобрести новогодние сувениры и сладости.'
log_prob, N = compute_join_trigram(phrase, bigrams_news, trigrams_news)
perplexity(log_prob, N)

53089.03380543496

In [263]:
phrase = '"В определенную службу оператора приходят запросы от департамента по чрезвычайным ситуациям, \
который информирует жителей различных регионов об изменениях в погодных условиях. <…> Отправителем данных является 112, \
мы выступаем как инструмент передачи сообщений. В данном конкретном случае произошла техническая ошибка. В рассылке значок \
градуса (°) был заменен на ноль. После обнаружения некорректной цифры всем абонентам региона, \
куда пришли сильные морозы, было отправлено исправленное сообщение", — пояснили в компании.'
log_prob, N = compute_join_trigram(phrase, bigrams_news, trigrams_news)
perplexity(log_prob, N)

33261.04571676868